# Libraries

In [151]:
import sys
pathModulesES = '../sauceforyall/'
sys.path.append(pathModulesES)

**Elasticsearch Query**

In [152]:
from yelpquery import YelpQuery
from pandasticsearch import Select
ye = YelpQuery()

**Machine Learning**

In [153]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

**Visualize**

In [154]:
import matplotlib.pyplot as plt
import seaborn as sns

**Others**

In [155]:
import re
import string
import pandas as pd

**Index name**

In [156]:
index_business = "yelp-business*"
index_review = "yelp-review*"
index_tip = "yelp-tip*"
index_user = "yelp-user*"

# 1. Data Pre-Processing

## 1.1 Load data

Retrieve all the reviews from the last 3 years, this helps reduce the volume of data

# **CHOOSE UR CITY**

In [157]:
city = "Las Vegas"

In [158]:
df_business = pd.read_json("../dataset/business.json", lines=True)
df_review = pd.read_json("../dataset/cleaned/review_cleaned_2016_2019.json", lines=True)

In [159]:
df_business.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,1SWheh84yJXfytovILXOAQ,Arizona Biltmore Golf Club,2818 E Camino Acequia Drive,Phoenix,AZ,85016,33.522143,-112.018481,3.0,5,0,{'GoodForKids': 'False'},"Golf, Active Life",None
1,QXAEGFB4oINsVuTFxEYKFQ,Emerald Chinese Restaurant,30 Eglinton Avenue W,Mississauga,ON,L5R 3E7,43.605499,-79.652289,2.5,128,1,"{'RestaurantsReservations': 'True', 'GoodForMe...","Specialty Food, Restaurants, Dim Sum, Imported...","{'Monday': '9:0-0:0', 'Tuesday': '9:0-0:0', 'W..."
2,gnKjwL_1w79qoiV3IC_xQQ,Musashi Japanese Restaurant,"10110 Johnston Rd, Ste 15",Charlotte,NC,28210,35.092564,-80.859132,4.0,170,1,"{'GoodForKids': 'True', 'NoiseLevel': 'u'avera...","Sushi Bars, Restaurants, Japanese","{'Monday': '17:30-21:30', 'Wednesday': '17:30-..."
3,xvX2CttrVhyG2z1dFg_0xw,Farmers Insurance - Paul Lorenz,"15655 W Roosevelt St, Ste 237",Goodyear,AZ,85338,33.455613,-112.395596,5.0,3,1,None,"Insurance, Financial Services","{'Monday': '8:0-17:0', 'Tuesday': '8:0-17:0', ..."
4,HhyxOkGAM07SRYtlQ4wMFQ,Queen City Plumbing,"4209 Stuart Andrew Blvd, Ste F",Charlotte,NC,28217,35.190012,-80.887223,4.0,4,1,"{'BusinessAcceptsBitcoin': 'False', 'ByAppoint...","Plumbing, Shopping, Local Services, Home Servi...","{'Monday': '7:0-23:0', 'Tuesday': '7:0-23:0', ..."


In [160]:
df_business =  df_business[["business_id","name","state","city","categories","attributes","is_open"]]
df_business = df_business[df_business.city == city]

In [161]:
df_business["attributes"].iloc[1]

{'OutdoorSeating': 'False',
 'BusinessAcceptsCreditCards': 'True',
 'RestaurantsDelivery': 'False',
 'RestaurantsReservations': 'True',
 'RestaurantsAttire': "'casual'",
 'Ambience': "{'romantic': True, 'intimate': False, 'touristy': False, 'hipster': False, 'divey': False, 'classy': True, 'trendy': False, 'upscale': False, 'casual': False}",
 'HasTV': 'False',
 'BYOBCorkage': "'no'",
 'NoiseLevel': "u'quiet'",
 'RestaurantsTakeOut': 'True',
 'RestaurantsPriceRange2': '2',
 'RestaurantsGoodForGroups': 'True',
 'WiFi': "u'no'",
 'Caters': 'True',
 'GoodForKids': 'True',
 'Alcohol': "u'full_bar'",
 'BusinessParking': "{'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}"}

In [162]:
import ast
def toList(dictt):
    l=[]
    if dict != None:
        for (idx,val) in dictt.items():
            if val == "True":
                l.append(idx) 
            elif idx == "Ambience":
                x = ast.literal_eval(val)
                if x != None:
                    for (idx2,val2) in x.items():
                        if val2 == True:
                            l.append(idx+"_"+idx2)
            elif idx == "BusinessParking": 
                x = ast.literal_eval(val)
                if x != None:
                    for (idx2,val2) in x.items():
                        if val2 == True:
                            l.append(idx+"_"+idx2)
            elif idx == "GoodForMeal":  
                x = ast.literal_eval(val)
                if x != None:
                    for (idx2,val2) in x.items():
                        if val2 == True:
                            l.append(idx+"_"+idx2)
            elif idx == "Music":  
                x = ast.literal_eval(val)
                if x != None:
                    for (idx2,val2) in x.items():
                        if val2 == True:
                            l.append(idx+"_"+idx2)
            elif idx == "BestNights":  
                x = ast.literal_eval(val)
                if x != None:
                    for (idx2,val2) in x.items():
                        if val2 == True:
                            l.append(idx+"_"+idx2)
            elif "u'" in val:
                l.append(idx+"_"+val[2:].replace("'",""))
            elif val != "False":
                l.append(idx+"_"+val.replace("'",""))
    return l

def fusion_attr(key, val):
    attr_full = ""
    key = key.replace("'","")
    sub_string = re.search('u\'(\w*)\'', val)
    if sub_string:
        attr_full = key + "_" + sub_string.group(1)
    else:
        val = val.replace("'","")
        attr_full = key + "_" + val
    return attr_full
    
def flatten_attr(nested_field):
    l=[]
    if nested_field:
        for (idx,val) in nested_field.items():
            f_level = idx
            s_level = re.findall(r"{(.+)}", val)    
            if s_level:
                for element in s_level[0].split(", "):
                    element_splited = element.split(":")
                    key_element = f_level + "_" + element_splited[0].strip()
                    val_element = element_splited[1].strip()
                    to_add_string = fusion_attr(key_element, val_element)
                    l.append(to_add_string)
            else:
                to_add_string = fusion_attr(f_level, val)
                l.append(to_add_string)    
    return ", ".join(l)


df_business['attributes'] = df_business['attributes'].map(lambda x: flatten_attr(x))

df_review = df_review[["business_id","user_id","stars","text","date"]]
final_df = pd.merge(df_business,df_review,how="inner", on='business_id') 
final_df.head()

,business_id,name,state,city,categories,attributes,is_open,user_id,stars,text,date
0,PZ-LZzSlhSe9utkQYU8pFg,Carluccio's Tivoli Gardens,NV,Las Vegas,"Restaurants, Italian","OutdoorSeating_False, BusinessAcceptsCreditCar...",0,BDjiEmXljD2ZHT61Iv9rrQ,5,I do not realize this place shutdown. I use to...,2016-08-09 19:02:44
1,nh_kQ16QAoXWwqZ05MPfBQ,Myron Hensel Photography,NV,Las Vegas,"Event Planning & Services, Photographers, Prof...",BusinessAcceptsCreditCards_True,1,_kS0Ww8UTuK1gj2gEIGPiw,5,"About a month ago, we have the pleasure of wor...",2016-08-06 16:25:52
2,nh_kQ16QAoXWwqZ05MPfBQ,Myron Hensel Photography,NV,Las Vegas,"Event Planning & Services, Photographers, Prof...",BusinessAcceptsCreditCards_True,1,9CsKkanhOGmTPpQXLeSwNg,5,I really could not have ask for a well photogr...,2016-06-23 18:50:13
3,nh_kQ16QAoXWwqZ05MPfBQ,Myron Hensel Photography,NV,Las Vegas,"Event Planning & Services, Photographers, Prof...",BusinessAcceptsCreditCards_True,1,Dvl_86XAXIquxV_ufN5JTg,5,Myron be our photographer at our wedding last ...,2016-06-05 06:27:37
4,nh_kQ16QAoXWwqZ05MPfBQ,Myron Hensel Photography,NV,Las Vegas,"Event Planning & Services, Photographers, Prof...",BusinessAcceptsCreditCards_True,1,ylFtcmpX7_u88xSmL0SjjA,5,Me and my husband get married a couple week ag...,2017-07-23 12:57:55


In [163]:
final_df.attributes

0          OutdoorSeating_False, BusinessAcceptsCreditCar...
1                            BusinessAcceptsCreditCards_True
2                            BusinessAcceptsCreditCards_True
3                            BusinessAcceptsCreditCards_True
4                            BusinessAcceptsCreditCards_True
                                 ...                        
1041550    OutdoorSeating_True, WiFi_free, RestaurantsPri...
1041551    OutdoorSeating_True, WiFi_free, RestaurantsPri...
1041552          BikeParking_False, RestaurantsPriceRange2_1
1041553          BikeParking_False, RestaurantsPriceRange2_1
1041554          BikeParking_False, RestaurantsPriceRange2_1
Name: attributes, Length: 1041555, dtype: object

In [164]:
final_df.to_json("../dataset/cleaned/review_stars_"+city+"_2016_2019.json", orient='records', lines=True)